In [ ]:
pip install matplotlib

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
import random
import math
import csv
import cv2
import os
# Modelin tahmin doğruluğunu değerlendirmek için karmaşıklık matrisini hesaplıyoruz.
from sklearn.metrics import classification_report, confusion_matrix
# Veriyi eğitim ve test setlerine ayırıyoruz. Modelin genelleştirme yeteneğini test etmek için önemlidir.
from sklearn.model_selection import train_test_split
# Etiketleri sayısal değerlere çeviriyoruz. Kategorik sınıfları modelin anlayacağı hale getiriyoruz.
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Activation
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import itertools
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Resimlerin okunması, resim ve etiket dizilerinin oluşturulma  aşaması
inputBasePath = r"C:\Users\İbrahim Demir\Desktop\Yeni klasör\images"
outputBasePath = r"C:\Users\İbrahim Demir\Desktop\Yeni klasör\imagearrays"
image_width = 224
image_height = 224
classes = ['cat', 'dog']
# Çalışma dizinini değiştiriyoruz, böylece veri setine erişim kolay olur.
os.chdir(inputBasePath)

X = []  # Resimleri tutmak için
Y = []  # Etiketleri tutmak için

for class1 in classes:
# Çalışma dizinini değiştiriyoruz, böylece veri setine erişim kolay olur.
    os.chdir(class1)
    print('=> ' + class1)
    for files in os.listdir('./'):
# Görüntüyü okuyoruz ve yeniden boyutlandırıyoruz, çünkü model belirli bir giriş boyutu bekler.
        img = cv2.imread(files)
        img = cv2.resize(img, (image_width, image_height))
        X.append(img)
        Y.append(class1)
# Çalışma dizinini değiştiriyoruz, böylece veri setine erişim kolay olur.
    os.chdir('..')

print("X : ", len(X))
print("Y : ", len(Y))

X = np.array(X).reshape(-1, image_width, image_height, 3)
Y = np.array(Y)

print("X : ", X.shape)
print("Y : ", Y.shape)

# Çalışma dizinini değiştiriyoruz, böylece veri setine erişim kolay olur.
os.chdir(outputBasePath)
np.save(str(image_width) + 'x' + str(image_height) + '_images', X)
np.save(str(image_width) + 'x' + str(image_height) + '_labels', Y)

print("[ INFO - STAGE1 ]  NUMPY ARRAY CREATION COMPLETED \n ")

# Sınıflandırma işlemleri
data = np.load(r"C:\Users\İbrahim Demir\Desktop\Yeni klasör\imagearrays\\224x224_images.npy")
labels = np.load(r"C:\Users\İbrahim Demir\Desktop\Yeni klasör\imagearrays\\224x224_labels.npy")

# Label Encoding
# Etiketleri sayısal değerlere çeviriyoruz. Kategorik sınıfları modelin anlayacağı hale getiriyoruz.
labelEn = LabelEncoder()
labels = labelEn.fit_transform(labels)
labels = to_categorical(labels)

# Train - Test Split
# Veriyi eğitim ve test setlerine ayırıyoruz. Modelin genelleştirme yeteneğini test etmek için önemlidir.
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, shuffle=True)

print(
"""
x_train shape: {}
x_test shape: {}
y_train shape: {}
y_test shape: {}
""".format(x_train.shape, x_test.shape, y_train.shape, y_test.shape))

# Normalizasyon
x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)
x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_test_mean) / x_test_std

# Train - Validation Split
# Veriyi eğitim ve test setlerine ayırıyoruz. Modelin genelleştirme yeteneğini test etmek için önemlidir.
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.10, shuffle=True, random_state=42)

# Model tanımlama
def model1(input_shape=(image_width, image_height, 3), num_classes=2):
    model = Sequential()
    chanDim = -1

# Evrişim katmanı ekleniyor. Görüntüden anlamlı özellikleri çıkarmak için kullanılır.
    model.add(Conv2D(64, (3, 3), padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
# Evrişim katmanı ekleniyor. Görüntüden anlamlı özellikleri çıkarmak için kullanılır.
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
# Havuzlama katmanı ekleniyor. Özellik haritasının boyutunu küçültüp hesaplama maliyetini azaltır.
    model.add(MaxPooling2D(pool_size=(2, 2)))

# Evrişim katmanı ekleniyor. Görüntüden anlamlı özellikleri çıkarmak için kullanılır.
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
# Evrişim katmanı ekleniyor. Görüntüden anlamlı özellikleri çıkarmak için kullanılır.
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
# Havuzlama katmanı ekleniyor. Özellik haritasının boyutunu küçültüp hesaplama maliyetini azaltır.
    model.add(MaxPooling2D(pool_size=(2, 2)))

# Evrişim katmanı ekleniyor. Görüntüden anlamlı özellikleri çıkarmak için kullanılır.
    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
# Evrişim katmanı ekleniyor. Görüntüden anlamlı özellikleri çıkarmak için kullanılır.
    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
# Havuzlama katmanı ekleniyor. Özellik haritasının boyutunu küçültüp hesaplama maliyetini azaltır.
    model.add(MaxPooling2D(pool_size=(2, 2)))

# Evrişim katmanı ekleniyor. Görüntüden anlamlı özellikleri çıkarmak için kullanılır.
    model.add(Conv2D(512, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
# Evrişim katmanı ekleniyor. Görüntüden anlamlı özellikleri çıkarmak için kullanılır.
    model.add(Conv2D(512, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
# Havuzlama katmanı ekleniyor. Özellik haritasının boyutunu küçültüp hesaplama maliyetini azaltır.
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
# Tam bağlı katman ekleniyor. Özellikleri kullanarak sınıflandırma yapacak nöronlar.
    model.add(Dense(4096))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

# Tam bağlı katman ekleniyor. Özellikleri kullanarak sınıflandırma yapacak nöronlar.
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))

    return model

model = model1()
model.summary()

# Optimizer tanımlama
optimizer = Adam(learning_rate=0.0001)

# LR annealer tanımlama
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

# Modeli derle
# Modeli derliyoruz. Optimizasyon fonksiyonunu, kayıp fonksiyonunu ve metrikleri belirtiyoruz.
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Epoch ve batch size tanımla
epc = 5 
bs = 50

# Modeli çalıştır
# Modeli eğitiyoruz. Verileri kullanarak ağırlıkları güncelleyerek öğrenme sağlanıyor.
history = model.fit(x_train, y_train, batch_size=bs, epochs=epc, validation_data=(x_validate, y_validate), verbose=1, callbacks=[learning_rate_reduction])

# Doğruluk grafiklerini çiz
# Eğitim sürecinin başarımını görselleştirmek için grafik çiziyoruz.
plt.plot(history.history['accuracy'])
# Eğitim sürecinin başarımını görselleştirmek için grafik çiziyoruz.
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Hata grafiklerini çiz
# Eğitim sürecinin başarımını görselleştirmek için grafik çiziyoruz.
plt.plot(history.history['loss'])
# Eğitim sürecinin başarımını görselleştirmek için grafik çiziyoruz.
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Confusion Matrix
Y_pred = model.predict(x_validate)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(y_validate, axis=1)

# Modelin tahmin doğruluğunu değerlendirmek için karmaşıklık matrisini hesaplıyoruz.
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

# Modelin tahmin doğruluğunu değerlendirmek için karmaşıklık matrisini hesaplıyoruz.
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Sınıf isimlerini almak için
class_names = labelEn.inverse_transform(range(len(classes)))
# Modelin tahmin doğruluğunu değerlendirmek için karmaşıklık matrisini hesaplıyoruz.
plot_confusion_matrix(confusion_mtx, classes=class_names)

# Modeli kaydetme
# Çalışma dizinini değiştiriyoruz, böylece veri setine erişim kolay olur.
os.chdir(r"C:\Users\İbrahim Demir\Desktop\Yeni klasör\models")
# Eğitilmiş modeli kaydediyoruz, böylece tekrar eğitmeye gerek kalmadan kullanabiliriz.
model.save('model1.h5')

# Veri Arttırımı (Data Augmentasyonu)
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)
datagen.fit(x_train)

# Veri arttırımı ile birlikte modeli fit et
# Modeli eğitiyoruz. Verileri kullanarak ağırlıkları güncelleyerek öğrenme sağlanıyor.
history = model.fit(datagen.flow(np.array(x_train), np.array(y_train), batch_size=bs), 
                    epochs=epc, 
                    validation_data=datagen.flow(np.array(x_validate), np.array(y_validate), batch_size=bs), 
                    verbose=1, 
                    steps_per_epoch=x_train.shape[0] // bs, 
                    callbacks=[learning_rate_reduction])

# Sonuçların yazdırılması
Y_pred = model.predict(x_test)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(y_test, axis=1)

# Sınıflandırma raporunu yazdırma
print("Sınıflandırma Raporu:")
print(classification_report(Y_true, Y_pred_classes, target_names=class_names))

# Karmaşıklık matrisini yazdırma
# Modelin tahmin doğruluğunu değerlendirmek için karmaşıklık matrisini hesaplıyoruz.
confusion_mtx_test = confusion_matrix(Y_true, Y_pred_classes)
plt.figure(figsize=(8, 6))
# Modelin tahmin doğruluğunu değerlendirmek için karmaşıklık matrisini hesaplıyoruz.
plot_confusion_matrix(confusion_mtx_test, classes=class_names)
plt.title('Test Seti Karmaşıklık Matrisi')
plt.show()

train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
print(f"Eğitim Doğruluğu: {train_acc:.4f}")
print(f"Doğrulama Doğruluğu: {val_acc:.4f}")